# Portfolio Project - Web Scraping Football Matches From The EPL in Python

### Introduction

This project is a first part of a Machine Learning project where I will predict the winner of each football match on the English Premier League (EPL).
In this project the main goal is gathering data for a cleaned pandas dataframe via web scraping.

### Downloading and Exploring the EPL Stats Page

In [1]:
import requests

In [2]:
url = "https://fbref.com/en/comps/9/2021-2022/2021-2022-Premier-League-Stats"

In [3]:
page = requests.get(url)

In [4]:
page.status_code # if the code is 200 then OK

200

Inspecting the page in web browser I can see the `id` of the table-tag which contains the teams and theirs results is `results2021-202291_overall`. Furthermore the HTML Tag which contains the Team name and their URL to details is `team`.

### Parsing HTML Links

In [5]:
from bs4 import BeautifulSoup

In [6]:
soup = BeautifulSoup(page.content, "html.parser")

In [7]:
teams_table = soup.find(id="results2021-202291_overall")

In [8]:
links = teams_table.find_all("a")

In [9]:
"""
Example links:

Link: /en/squads/19538871/2021-2022/Manchester-United-Stats, Text: Manchester Utd    - if contains word 'squads' then it refer to club
Link: /en/players/dea698d9/Cristiano-Ronaldo, Text: Cristiano Ronaldo
"""

club_paths = []

for link in links:
    path = link.get("href")
    
    if ("squads") in path:
        club_paths.append(path)

In [10]:
club_paths

['/en/squads/b8fd03ef/2021-2022/Manchester-City-Stats',
 '/en/squads/822bd0ba/2021-2022/Liverpool-Stats',
 '/en/squads/cff3d9bb/2021-2022/Chelsea-Stats',
 '/en/squads/361ca564/2021-2022/Tottenham-Hotspur-Stats',
 '/en/squads/18bb7c10/2021-2022/Arsenal-Stats',
 '/en/squads/19538871/2021-2022/Manchester-United-Stats',
 '/en/squads/7c21e445/2021-2022/West-Ham-United-Stats',
 '/en/squads/a2d435b3/2021-2022/Leicester-City-Stats',
 '/en/squads/d07537b9/2021-2022/Brighton-and-Hove-Albion-Stats',
 '/en/squads/8cec06e1/2021-2022/Wolverhampton-Wanderers-Stats',
 '/en/squads/b2b47a98/2021-2022/Newcastle-United-Stats',
 '/en/squads/47c64c55/2021-2022/Crystal-Palace-Stats',
 '/en/squads/cd051869/2021-2022/Brentford-Stats',
 '/en/squads/8602292d/2021-2022/Aston-Villa-Stats',
 '/en/squads/33c895d4/2021-2022/Southampton-Stats',
 '/en/squads/d3fd31cc/2021-2022/Everton-Stats',
 '/en/squads/5bfb9659/2021-2022/Leeds-United-Stats',
 '/en/squads/943e8050/2021-2022/Burnley-Stats',
 '/en/squads/2abfe087/2021-

In [30]:
club_urls = ["https://fbref.com" + path for path in club_paths] # Creating a list of full URLs (domain url + paths of each club)

In [31]:
club_urls

['https://fbref.com/en/squads/b8fd03ef/2021-2022/Manchester-City-Stats',
 'https://fbref.com/en/squads/822bd0ba/2021-2022/Liverpool-Stats',
 'https://fbref.com/en/squads/cff3d9bb/2021-2022/Chelsea-Stats',
 'https://fbref.com/en/squads/361ca564/2021-2022/Tottenham-Hotspur-Stats',
 'https://fbref.com/en/squads/18bb7c10/2021-2022/Arsenal-Stats',
 'https://fbref.com/en/squads/19538871/2021-2022/Manchester-United-Stats',
 'https://fbref.com/en/squads/7c21e445/2021-2022/West-Ham-United-Stats',
 'https://fbref.com/en/squads/a2d435b3/2021-2022/Leicester-City-Stats',
 'https://fbref.com/en/squads/d07537b9/2021-2022/Brighton-and-Hove-Albion-Stats',
 'https://fbref.com/en/squads/8cec06e1/2021-2022/Wolverhampton-Wanderers-Stats',
 'https://fbref.com/en/squads/b2b47a98/2021-2022/Newcastle-United-Stats',
 'https://fbref.com/en/squads/47c64c55/2021-2022/Crystal-Palace-Stats',
 'https://fbref.com/en/squads/cd051869/2021-2022/Brentford-Stats',
 'https://fbref.com/en/squads/8602292d/2021-2022/Aston-Vill

### Extracting Match Stats

In [35]:
page = requests.get(club_urls[5]) # Get the stats of Manchester United ( My favourite club :) )

In [36]:
page.status_code

200

In [37]:
soup = BeautifulSoup(page.content, 'html.parser')

In [40]:
mu_stats = soup.find(id="matchlogs_for") # ID of table that contains the "Scores & Fixtures" is "match_for_logs"

In [75]:
# Extracting stats from table called "Scores & Fixtures" 

columns = [] # It will be the column names of the DataFrame.

for name in mu_stats.find_all("tr")[0]:
    if name.string != " ": # It stores a space between elements. I will just ommit them.
        columns.append(name.string)

match_stats = []

for match in mu_stats.find_all("tr")[1:]: # Ommiting the header (first "tr" tag)
    stats = []
    
    for stat in match:
        if stat.find("span"): # The time data is inner a "span" tag within "td" tag
            stats.append(stat.find("span").text)
        else:
            stats.append(stat.string) # else I added the value os "td" tag to list

    match_stats.append(stats)

In [76]:
import pandas as pd

In [77]:
mu_table = pd.DataFrame(columns=columns, data=match_stats)

In [78]:
mu_table.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Opp Formation,Referee,Match Report,Notes
0,2021-08-14,12:30,Premier League,Matchweek 1,Sat,Home,W,5,1,Leeds United,1.5,0.5,49,"72,732",Harry Maguire,4-2-3-1,4-1-4-1,Paul Tierney,Match Report,None
1,2021-08-22,14:00,Premier League,Matchweek 2,Sun,Away,D,1,1,Southampton,1.8,0.7,63,"32,000",Harry Maguire,4-2-3-1,4-4-2,Craig Pawson,Match Report,None
2,2021-08-29,16:30,Premier League,Matchweek 3,Sun,Away,W,1,0,Wolves,0.6,2.1,56,"30,621",Harry Maguire,4-2-3-1,3-4-3,Mike Dean,Match Report,None
3,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Home,W,4,1,Newcastle Utd,2.5,0.4,63,"72,732",Harry Maguire,4-2-3-1,5-4-1,Anthony Taylor,Match Report,None
4,2021-09-14,18:45,Champions Lg,Group stage,Tue,Away,L,1,2,ch,0.5,1.4,46,"31,120",Harry Maguire,4-2-3-1,4-2-3-1,François Letexier,Match Report,None
